In [1]:
import requests
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import boto3
from datetime import datetime
import os
from io import BytesIO

In [2]:
link2 = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad"

In [3]:
link2

'https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad'

In [4]:
requisicao = requests.get(link2)

In [5]:
requisicao.json()

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02n'}],
 'base': 'stations',
 'main': {'temp': 291.4,
  'feels_like': 291.5,
  'temp_min': 290.37,
  'temp_max': 293.01,
  'pressure': 1018,
  'humidity': 85,
  'sea_level': 1018,
  'grnd_level': 1014},
 'visibility': 10000,
 'wind': {'speed': 0.74, 'deg': 105, 'gust': 1.76},
 'clouds': {'all': 11},
 'dt': 1697850097,
 'sys': {'type': 2,
  'id': 2020383,
  'country': 'BR',
  'sunrise': 1697791294,
  'sunset': 1697837809},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}

In [6]:
df = requisicao.json()

In [7]:
df1 = pd.json_normalize(df)

In [8]:
df_lon_lat = pd.DataFrame(df['coord'], index=[0])

In [9]:
df_lon_lat

,lon,lat
0,-51.1306,-29.6783


In [10]:
df_weather = pd.DataFrame(df['weather'], index=[0])

In [11]:
df_weather

,id,main,description,icon
0,801,Clouds,few clouds,02n


In [12]:
df_main = pd.DataFrame(df['main'], index=[0])

In [13]:
df_main

,temp,feels_like,temp_min,temp_max,pressure,humidity,sea_level,grnd_level
0,291.4,291.5,290.37,293.01,1018,85,1018,1014


In [14]:
df1 = pd.DataFrame(df1)

In [15]:
df1

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 801, 'main': 'Clouds', 'description': ...",stations,10000,1697850097,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1014,0.74,105,1.76,11,2,2020383,BR,1697791294,1697837809


In [16]:
selected_columns = [
    "base",
    "visibility",
    "dt",
    "timezone",
    "id",
    "name",
    "cod",
    "coord.lon",
    "coord.lat",
    "main.temp",
    "main.feels_like",
    "main.temp_min",
    "main.temp_max",
    "main.pressure",
    "main.humidity",
    "main.sea_level",
    "main.grnd_level",
    "wind.speed",
    "wind.deg",
    "wind.gust",
    "clouds.all",
    "sys.type",
    "sys.id",
    "sys.country",
    "sys.sunrise",
    "sys.sunset",
]

In [17]:
df_clumns_selected = df1[selected_columns]

In [18]:
df_clumns_selected

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,stations,10000,1697850097,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,291.4,...,1014,0.74,105,1.76,11,2,2020383,BR,1697791294,1697837809


In [19]:
super_df = pd.concat([df_weather, df_lon_lat, df_main, df_clumns_selected], axis=0, ignore_index=True)

In [20]:
super_df.to_parquet("C:/Users/crist/Desktop/s3_testes/super_df.parquet")

In [21]:
super_df

,id,main,description,icon,lon,lat,temp,feels_like,temp_min,temp_max,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,801.0,Clouds,few clouds,02n,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,-51.1306,-29.6783,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,291.4,291.5,290.37,293.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3456068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1014.0,0.74,105.0,1.76,11.0,2.0,2020383.0,BR,1.697791e+09,1.697838e+09


putaria com o boto3

In [22]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'

s3 = boto3.session.Session(profile_name=profile_name).client('s3')

In [23]:
bucket_name = 'cristian-teste-buckt-from-python-code'

In [24]:
local_file_path = "C:/Users/crist/Desktop/s3_testes/super_df.parquet"

In [25]:
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [26]:
dia_da_exucucao

'2023-10-20-22-02-52'

In [27]:
s3_file_name = f'df_teste_{dia_da_exucucao}.parquet'

In [28]:
s3_file_name

'df_teste_2023-10-20-22-02-52.parquet'

In [29]:
s3.upload_file(local_file_path, bucket_name, s3_file_name)

---------------------------------

In [30]:
bucket_name1 = 'cristian-teste-buckt-from-python-code'
nome_do_arquivo = 'df_teste_2023-10-15.parquet'  # Substitua pelo nome do arquivo que deseja ler


In [31]:
response = s3.get_object(Bucket=bucket_name1, Key=nome_do_arquivo)
conteudo_do_arquivo = response['Body'].read()


In [32]:
buffer = BytesIO(conteudo_do_arquivo)

In [33]:
table = pq.read_table(buffer)

In [34]:
df = table.to_pandas()

In [35]:
df

,id,main,description,icon,lon,lat,temp,feels_like,temp_min,temp_max,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,802.0,Clouds,scattered clouds,03n,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,NaN,None,None,None,-51.1306,-29.6783,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,NaN,None,None,None,NaN,NaN,289.14,288.94,288.7,290.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
3,3456068.0,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,...,1016.0,1.35,66.0,2.19,29.0,2.0,2020383.0,BR,1.697360e+09,1.697406e+09
